In [6]:
import pandas as pd

df = pd.read_csv("../data/raw/NSE_data_all_stocks_2024.csv")
print(df.columns)
print(df.iloc[:5])


Index(['Date', 'Code', 'Name', '12m Low', '12m High', 'Day Low', 'Day High',
       'Day Price', 'Previous', 'Change', 'Change%', 'Volume',
       'Adjusted Price'],
      dtype='object')
       Date  Code                     Name 12m Low 12m High Day Low Day High  \
0  2-Jan-24  EGAD              Eaagads Ltd   10.35     14.5    12.8     12.8   
1  2-Jan-24  KUKZ               Kakuzi Plc     342      440     385      385   
2  2-Jan-24  KAPC  Kapchorua Tea Kenya Plc     207      280     215      215   
3  2-Jan-24  LIMT           Limuru Tea Plc     365      380     380      380   
4  2-Jan-24  SASN               Sasini Plc    15.1       22      20       20   

  Day Price Previous Change Change%    Volume Adjusted Price  
0      12.8    13.95  -1.15  -8.24%       100              -  
1       385      385      -       -         -              -  
2       215      215      -       -         -              -  
3       380      380      -       -         -              -  
4        20     

In [8]:
import pandas as pd
import numpy as np
import os

# List of companies to keep
NSE_20 = [
    "EGAD", "KAPC", "KUKZ", "LIMT", "SASN", "WTK", "CGEN", "ABSA", "SBIC",
    "IMH", "DTK", "SCBK", "EQTY", "COOP", "BKG", "HFCK", "KCB", "NCBA",
    "XPRS", "SMER", "KQ", "NMG", "SGL", "TPSE", "SCAN", "UCHM", "LKL",
    "DCON", "NBV", "ARM", "BAMB", "CRWN", "CABL", "PORT", "TOTL", "KEGN",
    "KPLC", "UMME", "JUB", "SLM", "KNRE", "LBTY", "BRIT", "CIC", "OCH",
    "CTUM", "TCL", "HAFR", "KUVR", "NSE", "BOC", "BAT", "CARB", "EABL",
    "MSC", "UNGA", "EVRD", "AMAC", "FTGH", "SKL.O0000", "SCOM", "LAPR",
    "GLD", "SMWF.E0000"
]

# Paths
data_folder = "../data/raw"
processed_folder = "../data/processed"
os.makedirs(processed_folder, exist_ok=True)

# List all CSV files
csv_files = [f for f in os.listdir(data_folder) if f.endswith(".csv")]
print("Found CSVs:", csv_files)

# Columns to convert to numeric
numeric_cols = ['12m Low', '12m High', 'Day Low', 'Day High', 'Day Price', 'Previous', 'Change', 'Volume', 'Adjusted Price']

all_dfs = []

for file in csv_files:
    file_path = os.path.join(data_folder, file)
    df = pd.read_csv(file_path)
    
    # Standardize column headers
    df.columns = [col.strip().title() for col in df.columns]
    
    # Replace '-' with NaN
    df.replace('-', np.nan, inplace=True)
    
    # Convert numeric columns
    for col in numeric_cols:
        if col in df.columns:
            df[col] = df[col].astype(str).str.replace(',', '')
            df[col] = pd.to_numeric(df[col], errors='coerce')
    
    # Convert Date
    if 'Date' in df.columns:
        df['Date'] = pd.to_datetime(df['Date'], format='%d-%b-%y', errors='coerce')
    
    # Standardize Code and Name
    if 'Code' in df.columns:
        df['Code'] = df['Code'].str.upper()
    if 'Name' in df.columns:
        df['Name'] = df['Name'].str.title()
    
    all_dfs.append(df)

# Combine all years
combined_df = pd.concat(all_dfs, ignore_index=True)

# Fill missing Adjusted Price
combined_df['Adjusted Price'] = combined_df['Adjusted Price'].fillna(combined_df['Day Price'])

# Calculate missing Change%
def calc_change_pct(row):
    if pd.notnull(row['Day Price']) and pd.notnull(row['Previous']):
        return round((row['Day Price'] - row['Previous']) / row['Previous'] * 100, 2)
    return np.nan

combined_df['Change%'] = combined_df.apply(
    lambda row: calc_change_pct(row) if pd.isna(row['Change%']) else row['Change%'], axis=1
)

# Filter only NSE_20 companies
filtered_df = combined_df[combined_df['Code'].isin(NSE_20)].reset_index(drop=True)

# Save filtered dataset
output_file = os.path.join(processed_folder, "NSE_20_stocks_2013_2025_cleaned.csv")
filtered_df.to_csv(output_file, index=False)

print(f"✅ Filtered data for NSE_20 saved successfully to:\n{output_file}")


Found CSVs: ['NSE_data_all_stocks_2013.csv', 'NSE_data_all_stocks_2014.csv', 'NSE_data_all_stocks_2015.csv', 'NSE_data_all_stocks_2016.csv', 'NSE_data_all_stocks_2017.csv', 'NSE_data_all_stocks_2018.csv', 'NSE_data_all_stocks_2019.csv', 'NSE_data_all_stocks_2020_to_jun30.csv', 'NSE_data_all_stocks_2021_upto_31dec2021.csv', 'NSE_data_all_stocks_2022.csv', 'NSE_data_all_stocks_2023.csv', 'NSE_data_all_stocks_2024.csv', 'NSE_data_all_stocks_2025_jan_to_oct.csv']


C:\Users\HomePC\AppData\Local\Temp\ipykernel_19640\723393194.py:39: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df.replace('-', np.nan, inplace=True)
C:\Users\HomePC\AppData\Local\Temp\ipykernel_19640\723393194.py:39: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df.replace('-', np.nan, inplace=True)
C:\Users\HomePC\AppData\Local\Temp\ipykernel_19640\723393194.py:39: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_ob

✅ Filtered data for NSE_20 saved successfully to:
../data/processed\NSE_20_stocks_2013_2025_cleaned.csv


In [10]:
import pandas as pd

filtered_df = pd.read_csv("../data/processed/NSE_20_stocks_2013_2025_cleaned.csv")

# Show all columns with their repr to reveal hidden spaces
for col in filtered_df.columns:
    print(repr(col))


'Date'
'Code'
'Name'
'12M Low'
'12M High'
'Day Low'
'Day High'
'Day Price'
'Previous'
'Change'
'Change%'
'Volume'
'Adjust'
'Adjusted Price'


In [15]:
import pandas as pd
import numpy as np

# Load dataset
filtered_df = pd.read_csv("../data/processed/NSE_20_stocks_2013_2025_cleaned.csv")

# -------------------------
# Step 1: Clean column names
# -------------------------
filtered_df.columns = (
    filtered_df.columns
    .str.strip()
    .str.replace(' ', '_')
    .str.replace('%', 'pct')
    .str.lower()
)

# -------------------------
# Step 2: Convert numeric columns
# -------------------------
numeric_cols = ['12m_low', '12m_high', 'day_low', 'day_high', 'day_price', 'previous', 'change', 'volume', 'adjust', 'adjusted_price']

for col in numeric_cols:
    if col in filtered_df.columns:
        # Remove commas, convert to numeric, coerce errors to NaN
        filtered_df[col] = filtered_df[col].astype(str).str.replace(',', '').str.strip()
        filtered_df[col] = pd.to_numeric(filtered_df[col], errors='coerce')

# -------------------------
# Step 3: Feature creation
# -------------------------

# % distance from 12-month Low/High
filtered_df['pct_from_12m_low'] = ((filtered_df['day_price'] - filtered_df['12m_low']) / filtered_df['12m_low']) * 100
filtered_df['pct_from_12m_high'] = ((filtered_df['12m_high'] - filtered_df['day_price']) / filtered_df['12m_high']) * 100

# Daily return
filtered_df['daily_return'] = filtered_df['day_price'] - filtered_df['previous']

# Daily volatility
filtered_df['daily_volatility'] = filtered_df['day_high'] - filtered_df['day_low']

# Moving averages (5-day and 10-day) for each stock
filtered_df = filtered_df.sort_values(['code', 'date'])
filtered_df['ma_5'] = filtered_df.groupby('code')['day_price'].transform(lambda x: x.rolling(5).mean())
filtered_df['ma_10'] = filtered_df.groupby('code')['day_price'].transform(lambda x: x.rolling(10).mean())

# -------------------------
# Step 4: Save dataset with features
# -------------------------
output_file = "../data/processed/NSE_20_stocks_2013_2025_features.csv"
filtered_df.to_csv(output_file, index=False)

print(f"✅ Features created and saved successfully to:\n{output_file}")


✅ Features created and saved successfully to:
../data/processed/NSE_20_stocks_2013_2025_features.csv


In [16]:
import pandas as pd
from datetime import timedelta

# Load dataset already processed with features (from 1.3)
filtered_df = pd.read_csv("../data/processed/NSE_20_stocks_2013_2025_features.csv")

# -------------------------
# Step 1: Convert date to datetime
# -------------------------
filtered_df['date'] = pd.to_datetime(filtered_df['date'], errors='coerce', dayfirst=True)

# -------------------------
# Step 2: Create next-day target
# -------------------------
# Shift day_price by -1 per stock to get next-day price
filtered_df['next_day_price'] = filtered_df.groupby('code')['day_price'].shift(-1)

# Compute next-day return
filtered_df['next_day_return'] = filtered_df['next_day_price'] - filtered_df['day_price']

# Define Buy/Sell/Hold function
def buy_sell_hold(x, threshold=0.5):
    if x > threshold:
        return 'Buy'
    elif x < -threshold:
        return 'Sell'
    else:
        return 'Hold'

# Convert next-day return to % and apply function
filtered_df['target'] = (filtered_df['next_day_return'] / filtered_df['day_price'] * 100).apply(buy_sell_hold)

# Drop rows where next_day_price is NaN (last day of each stock)
filtered_df = filtered_df.dropna(subset=['next_day_price'])

# -------------------------
# Step 3: Filter last 30 days for dashboard
# -------------------------
end_date = filtered_df['date'].max()
start_date = end_date - timedelta(days=30)
filtered_last_month = filtered_df[(filtered_df['date'] >= start_date) & (filtered_df['date'] <= end_date)]

# -------------------------
# Step 4: Save datasets
# -------------------------
output_file_all = "../data/processed/NSE_20_stocks_2013_2025_features_target.csv"
output_file_last_month = "../data/processed/NSE_20_stocks_last_30days.csv"

filtered_df.to_csv(output_file_all, index=False)
filtered_last_month.to_csv(output_file_last_month, index=False)

print("✅ Step 1.4 complete:")
print(f" - Full dataset with target saved to: {output_file_all}")
print(f" - Last 30 days filtered dataset saved for dashboard: {output_file_last_month}")


✅ Step 1.4 complete:
 - Full dataset with target saved to: ../data/processed/NSE_20_stocks_2013_2025_features_target.csv
 - Last 30 days filtered dataset saved for dashboard: ../data/processed/NSE_20_stocks_last_30days.csv


In [17]:
import pandas as pd
from datetime import timedelta

# Load dataset with features + target from step 1.4
df = pd.read_csv("../data/processed/NSE_20_stocks_2013_2025_features_target.csv")

# -------------------------
# Step 1: Ensure date column is datetime
# -------------------------
df['date'] = pd.to_datetime(df['date'], errors='coerce', dayfirst=True)

# -------------------------
# Step 2: Sort by stock code and date
# -------------------------
df = df.sort_values(['code', 'date']).reset_index(drop=True)

# -------------------------
# Step 3: Prepare dataset for frontend/dashboard
# -------------------------
# Filter last 30 days for plotting in dashboard
end_date = df['date'].max()
start_date = end_date - timedelta(days=30)
df_last_30days = df[(df['date'] >= start_date) & (df['date'] <= end_date)]

# Optional: select only relevant columns for dashboard
dashboard_columns = [
    'date', 'code', 'name', 'day_price', 'ma_5', 'ma_10',
    'pct_from_12m_low', 'pct_from_12m_high', 'daily_return',
    'daily_volatility', 'target'
]
df_last_30days_dashboard = df_last_30days[dashboard_columns]

# -------------------------
# Step 4: Save the dashboard-ready dataset
# -------------------------
output_file_dashboard = "../data/processed/NSE_20_stocks_last_30days_dashboard.csv"
df_last_30days_dashboard.to_csv(output_file_dashboard, index=False)

print("✅ Step 1.5 complete:")
print(f" - Dashboard-ready last 30 days dataset saved: {output_file_dashboard}")


✅ Step 1.5 complete:
 - Dashboard-ready last 30 days dataset saved: ../data/processed/NSE_20_stocks_last_30days_dashboard.csv
